# Starbucks Promotional Offers Portfolio Project
**Main Objectives**
- Explore how different customer demographics (age, income, gender) engage with offers and how this impacts their purchasing power.
- Analyse the effectiveness of various offer types accross channels.

In [2]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

from ast import literal_eval
from sqlalchemy import create_engine

KeyboardInterrupt: 

## Importing the Data
I will use the pandas library to import the data into this workbook. I will then check both the head and tail of each table to ensure that they were loaded in correctly.

In [ ]:
# Load the data into pd.DataFrame objects
customers = pd.read_csv('customers.csv')
offers = pd.read_csv('offers.csv')
events = pd.read_csv('events.csv')

In [ ]:
# Make sure the offers table loaded in correctly by checking the head
offers.head(3)

In [ ]:
# Make sure the offers table loaded in correctly by checking the tail
offers.tail(3)

In [ ]:
# Make sure the customers table loaded in correctly by checking the head
customers.head(3)

In [ ]:
# Make sure the customers table loaded in correctly by checking the tail
customers.tail(3)

In [ ]:
# Make sure the events table loaded in correctly by checking the head
events.head(3)

In [ ]:
# Make sure the events table loaded in correctly by checking the tail
events.tail(3)

## Cleaning the Data
I will now clean each table in the dataset one by one. To do this, I will first make sure that any individual column in the table that is in an "unclean" format is put into a cleaner format. I will then investigate the data types of each column. At the same time, I will investigate the bit-sizes and potentail outliers of the numeric columns. I will make any necessary changes to these attributes. I will then investigate missing data and decide on an case-by-case basis what to do with the missing data. I will then specifically check if there are inconsitent text or typos in the categorical columns and make necessary fixes. I will then check for duplicated rows and decide whether it is necessary to delete them.
### Clean the `offers` Table
The `offers` table has a column called `channels` that is in the form of lists. Since Pandas does not work overally well with lists as values, I will create dummy variables for the individual channels.

In [ ]:
# Turn channels column into list type
offers['channels'] = offers['channels'].apply(literal_eval)

# Create channels dummies
for channel in ['web', 'email', 'mobile', 'social']:
    offers[channel] = offers['channels'].apply(lambda x: 1 if channel in x else 0).astype('int8')

# Drop the original columns list
offers = offers.drop('channels', axis=1)

In [ ]:
# Check the data types, size, and memory usage of the offers table
offers.info(memory_usage='deep')

In [ ]:
# Check the summary statistics for the numeric columns to see if we can reduce bit-sizes and identify possible outliers
offers.describe().round()

In [ ]:
# Reduce the bit-sizes of the numeric columns to save memory and optimise efficiency
offers = offers.astype({
    'difficulty': 'int8',
    'reward': 'int8',
    'duration': 'int8',
    'offer_type': 'category'
})

In [ ]:
# Check to see if the changes occured
offers.info(memory_usage='deep')

In [ ]:
# Check for any missing data
offers.isna().sum()

In [ ]:
# Check for any inconsistent text or typos, specifically in category columns
offers['offer_type'].unique()

In [ ]:
# Check for any duplicate rows
offers.duplicated().sum()

In [ ]:
# Here is the cleaned version of the offers table
offers.head(3)

### Cleaning the `customers` Table

In [ ]:
# Check the data types, size, and memory usage of the customers data frame
customers.info(memory_usage='deep')

In [ ]:
# Check the summary statistics for the numeric columns to see if we can reduce bit-sizes and identify possible outliers
customers.describe().round()

In [ ]:
# Check to see if gender is a categorical variable
customers['gender'].unique()

In [ ]:
# Convert the data types of the column in the wrong data type and reduce bit-sizes of numeric columns to save space
customers = customers.astype({
    'gender': 'category',
    'age': 'int16',
    'income': 'float32'
})

# Convert the became_member_on column to datetime dtype
customers['became_member_on'] = pd.to_datetime(customers['became_member_on'], format='%Y%m%d')

In [ ]:
# Check to see if the changes occured
customers.info(memory_usage='deep')

In [ ]:
# Check for any missing data
customers.isna().sum()

In [ ]:
# Investigate the rows that have missing values
customers_nan_rows = customers[customers.isna().any(axis=1)]
customers_nan_rows.head()

In [ ]:
# Check to see how many rows have values missing
len(customers_nan_rows)

In [ ]:
# Check the ages of all the missing values
customers_nan_rows['age'].unique()

In [ ]:
# Check how many 118 values there are in the age column
(customers['age'] == 118).sum()

In [ ]:
# Check the % of missing data
(len(customers_nan_rows) / len(customers)) * 100

In [ ]:
customers['age'].hist();

In `customers`, there are 2175 missing values each for both the `gender` and `income` columns. These missing values all occur in the same rows, meaning that there are 2175 rows that miss both the `gender` and `income` values. In addition, each one of these rows has `118` for its value for `age`, indicating that this is a placeholder for missing information. Moreover, it is skewing the distribution for the `age` column, so we will change these values to `np.NaN` values. Even though the rows with missing values make up 12.8% of our data, I will leave all the missing values as is for now so we do not lose any info when we join the tables later.

In [ ]:
# Replace the values of 118 in the age column with np.NaN values
customers['age'] = customers['age'].replace({118: np.NaN})

In [ ]:
# Check for dupilcate rows
customers.duplicated().sum()

### Cleaning the `events` Table
The `events` table has a column called `value` that is in the form of dictionaries. Since Pandas does not work overally well with dictionaries as values, I will seperate the keys and values into seperate columns.

In [ ]:
# Turn channels column into dict type
events['value'] = events['value'].apply(literal_eval)

# Create a series that holds the dictionary keys as a list
value_keys = events['value'].apply(lambda dict: list(dict.keys()))

# Create a series that holds the dictionary values as a list
value_values = events['value'].apply(lambda dict: list(dict.values()))

In [ ]:
# Check to see what values are in the dictionaries by checking the different possible keys.
value_keys.value_counts()

It looks like the `value` column has three different types of keys, which are `amount`, `offer_id`/`offer id`, and `reward`. Upon further investigation, the `value` values that have `amount` as their key are `transaction` events, the `value` values that just have `offer_id`/`offer id` as their key are `offer received` and `offer viewed` events, and `value` values that have both `offer_id` and `reward` as their keys are `offer completed` events. Hence, I will now create columns for these individual keys and corresponding values. However, for reward, I will just create a column called `reward` and fill it with the value or `NaN` 

In [ ]:
# Create a column that hold the value key (either amount or offer id/offer_id)
events['value_key'] = value_keys.apply(lambda lst: lst[0])

# Create a column that holds the value value (either the amount or the offer id/offer_id)
events['value'] = value_values.apply(lambda lst: lst[0])

#Create a column that holds the reward value if it exists
events['reward'] = value_values.apply(lambda lst: lst[1] if len(lst) == 2 else np.NaN)

In [ ]:
# Turn the time column into two columns that is the day number and hour of that day
events['day'] = events['time'] // 24
events['hour'] = events['time'] % 24

# Drop the original time column
events = events.drop('time', axis=1)

In [ ]:
# Check the data types, size, and memory usage of the events table
events.info(memory_usage='deep')

In [ ]:
# Check the summary statistics for the numeric columns to see if we can reduce bit-sizes and identify possible outliers
events.describe().round()

In [ ]:
# Check if the events column is categorical and if there is any inconsistent text or typos 
events['event'].value_counts()

In [ ]:
# Check if the events column is categorical and if there is any inconsistent text or typos 
events['value_key'].value_counts()

In [ ]:
# Fix the inconsitent text
events['value_key'] = events['value_key'].replace({'offer_id': 'offer id'})

In [ ]:
# Check to see if the inconsitent text was fixed
events['value_key'].value_counts()

In [ ]:
# Convert the data types of the column in the wrong data type and reduce bit-sizes of numeric columns to save space
events = events.astype({
    'event': 'category',
    'value_key': 'category',
    'reward': 'float32',
    'day': 'int8',
    'hour': 'int8'
})

In [ ]:
# Check to see if the changes occured
events.info(memory_usage='deep')

In [ ]:
# Check for missing values
events.isna().sum()

These missing values are a reflection of the amount of events in the table that are not `offer completed`. Therefore we want these values to remain missing.

In [ ]:
# Check for duplicate rows
events.duplicated().sum()

In [ ]:
# Investigate the duplicate rows
events_dup_rows = events[events.duplicated()]
events_dup_rows.head()

In [ ]:
# Investigate the event types of the duplicate rows
events_dup_rows['event'].unique()

In [ ]:
# Investigate the offer types of duplicate rows
events_dup_rows.merge(offers, how='inner', left_on='value', right_on='offer_id')['offer_type'].unique()

The only consitency with all the duplicate values in the `events` table is that they are all `offer completed` events. Therefore, I will leave them for now just incase another reason pops up when we join the tables later. <br>
<br>
Since the `events` table has has four distinct events. It seems benificial to break up the `events` table into the four distinct events and clean the now seperate tables accordingly.

In [ ]:
# Divide the events table into each event type
def divide_table(event):
    filtered_table = events[events['event'] == event]
    filtered_table = filtered_table.drop('event', axis=1)
    if event == 'offer completed':
        filtered_table = filtered_table.drop('value_key', axis=1).rename({'value': 'offer_id'}, axis=1)
    elif event == 'transaction':
        filtered_table = filtered_table.drop(['reward', 'value_key'], axis=1).rename({'value': 'amount'}, axis=1)
    else:
        filtered_table = filtered_table.drop(['reward', 'value_key'], axis=1).rename({'value': 'offer_id'}, axis=1)
    return filtered_table

offers_completed = divide_table('offer completed')
offers_received = divide_table('offer received')
offers_viewed = divide_table('offer viewed')
transactions = divide_table('transaction')

## Deporting Data to MySQL

**Fix The Following Code**

In [ ]:
user = 'root'
password = ''
host = 'localhost'
port = '3306'
database = 'starbucks_db'

In [ ]:
#engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Send the DataFrame to MySQL
#offers.to_sql('my_table', con=engine, if_exists='replace', index=False)